In [1]:
import math
import random

In [2]:
# pip install graphviz
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root, format='svg', rankdir='LR'):
    """
    format: png | svg | ...
    rankdir: TB (top to bottom graph) | LR (left to right)
    """
    assert rankdir in ['LR', 'TB']
    nodes, edges = trace(root)
    dot = Digraph(format=format, graph_attr={'rankdir': rankdir}) #, node_attr={'rankdir': 'TB'})
    
    for n in nodes:
        dot.node(name=str(id(n)), label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            dot.node(name=str(id(n)) + n._op, label=n._op)
            dot.edge(str(id(n)) + n._op, str(id(n)))
    
    for n1, n2 in edges:
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)
    
    return dot

In [3]:
class Value:
    """ stores a single scalar value and its gradient """

    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        # internal variables
        self._prev = set(_children)
        self._op = _op
        self._backward = lambda: None
        self.label = label

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other): # self + other
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        
        self._backward = _backward
        
        return out

    def __mul__(self, other): # self * other
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        self._backward = _backward
        
        return out

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting int & float powers"
        out = Value(math.pow(self.data, other), (self, ), 'pow')

        def _backward():
            self.grad += (other * (self.data ** (other - 1))) * out.grad
        
        self._backward = _backward
        
        return out

    def exp(self):
        out = Value(math.exp(self.data), (self, ), 'exp')

        def _backward():
            self.grad += out.data * out.grad
        
        self._backward = _backward
        
        return out

    def tanh(self):
        e = math.exp(2 * self.data)
        t = (e - 1) / (e + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward():
            self.grad += (1 - t**2) * out.grad
        
        self._backward = _backward
        
        return out

    def relu(self):
        out = Value(max(self.data, 0.0), (self, ), 'relu')

        def _backward():
            self.grad += (out.data > 0) * out.grad
        
        self._backward = _backward
        
        return out

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

    def __radd__(self, other): # other + self
        return self + other

    def __rmul__(self, other): # other * self
        return self * other

    def __neg__(self): # -self
        return -1.0 * self

    def __rsub__(self, other): # other - self
        return other + (-self)

    def __sub__(self, other): # self - other
        return self + (-other)

    def __rtruediv__(self, other): # other / self
        return other * (self ** -1)

    def __truediv__(self, other): # self / other
        return self * (other ** -1)

In [ ]:
# inputs: x1, x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights: w1, w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# x1*w1 + x2*w2 + b
x1w1 = x1 * w1; x1w1.label = 'x1w1'
x2w2 = x2 * w2; x2w2.label = 'x2w2'
b = Value(6.8813735870195, label='b')
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label = 'x1w1x2w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.relu(); o.label = 'o'

In [ ]:
draw_dot(o)

In [ ]:
o.backward()

In [4]:
class Module:

    def zero_grad(self):
        for p in self.parameters():
            p.grad = 0.0

    def parameters(self):
        return []

class Neuron(Module):

    def __init__(self, n_ins):
        self.w = [Value(random.uniform(-1, 1)) for _ in range(n_ins)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        act = sum([xi * wi for xi, wi in zip(x, self.w)], self.b)
        out = act.tanh()
        return act

    def parameters(self):
        return self.w + [self.b]

class Layer(Module):

    def __init__(self, n_in, n_out):
        self.neurons = [Neuron(n_in) for _ in range(n_out)]

    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for n in self.neurons for p in n.parameters()]

class MLP(Module):

    def __init__(self, n_in, n_outs):
        sz = [n_in] + n_outs
        self.layers = [Layer(sz[i], sz[i + 1]) for i in range(len(n_outs))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for l in self.layers for p in l.parameters()]

In [5]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0]
]
ys = [1.0, -1.0, -1.0, 1.0]
n = MLP(3, [4, 4, 1])

In [6]:
for itr in range(100):
    # forward pass
    ypreds = [n(x) for x in xs]

    # loss
    loss = sum((ypred - ytgt)**2 for ypred, ytgt in zip(ypreds, ys))
    print(f"itr: {itr}, loss: {loss.data}")

    # zero grad
    n.zero_grad()

    # backward
    loss.backward()

    # parameter update
    for p in n.parameters():
        p.data += (-0.1) * p.grad

itr: 0, loss: 23.008284180874877
itr: 1, loss: 15.44274587979545
itr: 2, loss: 7.9482040858849805
itr: 3, loss: 29.55912324131421
itr: 4, loss: 5.56638184090358
itr: 5, loss: 2.862866075511343
itr: 6, loss: 3.207801183152697
itr: 7, loss: 2.635607554937067
itr: 8, loss: 1.7533389047497792
itr: 9, loss: 1.3171221025928388
itr: 10, loss: 0.2635415588493433
itr: 11, loss: 0.36197596731430887
itr: 12, loss: 0.5864442138736787
itr: 13, loss: 1.3733459173896536
itr: 14, loss: 1.0366880379996768
itr: 15, loss: 0.41410658858565985
itr: 16, loss: 0.4036836472634331
itr: 17, loss: 0.09617201764959686
itr: 18, loss: 0.17949993673007136
itr: 19, loss: 0.12480593795585805
itr: 20, loss: 0.0652314299368495
itr: 21, loss: 0.07128988199611971
itr: 22, loss: 0.04282409378936507
itr: 23, loss: 0.06022795505597622
itr: 24, loss: 0.03895869961930819
itr: 25, loss: 0.01504205814630587
itr: 26, loss: 0.008037442882249364
itr: 27, loss: 0.010215537242594002
itr: 28, loss: 0.020701506124887834
itr: 29, loss: 

In [7]:
ypreds

[Value(data=0.999840993523653),
 Value(data=-0.9999614495332027),
 Value(data=-1.000164477187989),
 Value(data=1.0000928703215364)]

In [8]:
ys

[1.0, -1.0, -1.0, 1.0]